In [ ]:
from IPython.core.display import HTML
with open('style.css', 'r') as file:
    css = file.read()
HTML(css)

# Refutational Completeness of the Cut Rule

This notebook implements a number of procedures that are needed in our proof of the <em style="color:blue">refutational completeness</em> of the cut rule.

The function $\texttt{complement}(l)$ computes the <em style="color:blue">complement</em> of a literal $l$.
If $p$ is a propositional variable, we have the following: 
<ol>
    <li>$\texttt{complement}(p) = \neg p$,
    </li>
    <li>$\texttt{complement}(\neg p) = p$.
    </li>
</ol>

In [ ]:
def complement(l):
    "Compute the complement of the literal l."
    if isinstance(l, str):  # l is a propositional variable
        return ('¬', l)
    else:                   # l = ('¬', 'p')
        return l[1]         # l[1] = p

In [ ]:
complement('p')

In [ ]:
complement(('¬', 'p'))

The function $\texttt{extractVariable}(l)$ extracts the propositional variable from the literal $l$.
If $p$ is a propositional variable, we have the following: 
<ol>
    <li>$\texttt{extractVariable}(p) = p$,
    </li>
    <li>$\texttt{extractVariable}(\neg p) = p$.
    </li>
</ol>

In [ ]:
def extractVariable(l):
    "Extract the variable of the literal l."
    if isinstance(l, str):  # l is a propositional variable
        return l
    else:                   # l = ('¬', 'p')
        return l[1]

In [ ]:
extractVariable('p')

In [ ]:
extractVariable(('¬', 'p'))

The function $\texttt{collectsVariables}(M)$ takes a set of clauses $M$ as its input and computes the set of all propositional variables occurring in $M$.  The clauses in $M$ are represented as sets of literals.

In [ ]:
def collectVariables(M):
    "Return the set of all variables occurring in M."
    return { extractVariable(l) for C in M 
                                for l in C
           }

In [ ]:
C1 = frozenset({ 'p', 'q', 'r' })
C2 = frozenset({ ('¬', 'p'), ('¬', 'q'), ('¬', 's') })
collectVariables({C1, C2})

Given two clauses $C_1$ and $C_2$ that are represented as sets of literals, the function `cutRule`$(C_1, C_2)$ computes all clauses that can be derived from $C_1$ and $C_2$ using the *cut rule*.  In set notation, the cut rule is the following rule of inference:
$$
   \frac{\displaystyle \;C_1\cup \{l\} \quad C_2 \cup \bigl\{\overline{\,l\,}\bigr\}}{\displaystyle C_1 \cup C_2}
$$

In [ ]:
def cutRule(C1, C2):
    "Return the set of all clauses that can be deduced by the cut rule from C1 and C2."
    return { C1 - {l} | C2 - {complement(l) } for l in C1
                                              if  complement(l) in C2
           }

In [ ]:
C1 = frozenset({ 'p', 'q' })
C2 = frozenset({ ('¬', 'p'), ('¬', 'q') })
cutRule(C1, C2)

In the expression `saturate(Clauses)` below, `Clauses` is a set of *clauses*, where each clause is a set of *literals*.  The call `saturate(Clauses)` computes the set of all clauses that can be derived from clauses in the set `Clauses` using the *cut rule*.  The function keeps applying the cut rule until either no new clauses can be derived, or the empty clause $\{\}$ is derived.  The resulting set of clauses is *saturated* in the following sense:  If $C_1$ and $C_2$ are clauses from the set `Clauses` and the clause $D$ can be derived from $C_1$ and $C_2$ via the cut rule, then $D \in \texttt{Clauses}$ or $\{\} \in \texttt{Clauses}$.

In [ ]:
def saturate(Clauses):
    while True:
        Derived = { C for C1 in Clauses
                      for C2 in Clauses
                      for C  in cutRule(C1, C2)
                  }
        if frozenset() in Derived:
            return { frozenset() }  # This is the set notation of ⊥.
        Derived -= Clauses          # remove clauses that were present before
        if Derived == set():        # no new clauses have been found
            return Clauses
        Clauses |= Derived

In [ ]:
C1 = frozenset({ 'p', 'q' })
C2 = frozenset({ ('¬', 'p') })
C3 = frozenset({ ('¬', 'p'), ('¬', 'q') })
saturate({C1, C2, C3})

The function $\texttt{findValuation}(\texttt{Clauses})$ takes a set of clauses as input.  The function tries to compute a variable interpretation that makes all of the clauses true.  If this is successful, a set of literals is returned.  This set of literals does not contain  any complementary literals and therefore corresponds to a variable assignment satisfying all clauses.  If $\texttt{Clauses}$ is unsatisfiable, <tt>False</tt> is returned.

In [ ]:
def findValuation(Clauses):
    "Given a set of Clauses, find a propositional valuation satisfying all of these clauses."
    Variables = collectVariables(Clauses)
    Clauses   = saturate(Clauses)
    if frozenset() in Clauses:  # The set Clauses is inconsistent.
        return False
    Literals = set()
    for p in Variables:
        if any(C for C in Clauses 
                 if  p in C and C - {p} <= { complement(l) for l in Literals }
              ):
            Literals |= { p }
        else:
            Literals |= { ('¬', p) }
    return Literals

In [ ]:
C1 = frozenset({ 'r', 'p', 's' })
C2 = frozenset({ 'r', 's' })
C3 = frozenset({ 'p', 'q', 's' })
C4 = frozenset({ ('¬', 'p'), ('¬', 'q') })
C5 = frozenset({ ('¬', 'p'), 's', ('¬', 'r') })
C6 = frozenset({ 'p', ('¬', 'q'), 'r'})
C7 = frozenset({ ('¬', 'r'), ('¬', 's'), 'q' })
C8 = frozenset({ ('¬', 'p'), ('¬', 's')})
C9 = frozenset({ 'p', ('¬', 'r'), ('¬', 'q') })
C0 = frozenset({ ('¬', 'p'), 'r', 'q', ('¬', 's') })
Clauses  = { C0, C1, C2, C3, C4, C5, C6, C7, C8, C9 }
findValuation(Clauses)